In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [4]:
spark = SparkSession.builder.appName('abc').getOrCreate()

In [5]:
df = spark.read.json('ga_sample.json.gz')

In [6]:
df.printSchema()

root
 |-- channelGrouping: string (nullable = true)
 |-- customDimensions: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- index: string (nullable = true)
 |    |    |-- value: string (nullable = true)
 |-- date: string (nullable = true)
 |-- device: struct (nullable = true)
 |    |-- browser: string (nullable = true)
 |    |-- browserSize: string (nullable = true)
 |    |-- browserVersion: string (nullable = true)
 |    |-- deviceCategory: string (nullable = true)
 |    |-- flashVersion: string (nullable = true)
 |    |-- isMobile: boolean (nullable = true)
 |    |-- language: string (nullable = true)
 |    |-- mobileDeviceBranding: string (nullable = true)
 |    |-- mobileDeviceInfo: string (nullable = true)
 |    |-- mobileDeviceMarketingName: string (nullable = true)
 |    |-- mobileDeviceModel: string (nullable = true)
 |    |-- mobileInputSelector: string (nullable = true)
 |    |-- operatingSystem: string (nullable = true)
 |    |-- operati

In [7]:
df.cache().count()

2556

In [8]:
df.groupBy('device.browser').count().show()

+--------------------+-----+
|             browser|count|
+--------------------+-----+
|Mozilla Compatibl...|    1|
|   Internet Explorer|   54|
|             Firefox|  101|
|           YaBrowser|    2|
|              Safari|  397|
|       Nokia Browser|    2|
|             Coc Coc|    2|
|     Android Browser|    2|
|          Opera Mini|   21|
|     Android Webview|   19|
|     Safari (in-app)|   10|
|              Chrome| 1900|
|          UC Browser|    6|
|                Edge|   23|
|               Opera|   16|
+--------------------+-----+



In [9]:
df.groupBy('totals.transactionRevenue').count().show()

+------------------+-----+
|transactionRevenue|count|
+------------------+-----+
|         135070000|    1|
|        1000780000|    1|
|          30790000|    1|
|          48000000|    1|
|        2933610000|    1|
|          27430000|    1|
|          11960000|    1|
|         200000000|    1|
|         170390000|    1|
|          28790000|    1|
|          82480000|    1|
|         347140000|    1|
|              null| 2513|
|          27180000|    1|
|          33500000|    1|
|          67190000|    1|
|           2990000|    1|
|          29580000|    1|
|         144690000|    1|
|          44000000|    1|
+------------------+-----+
only showing top 20 rows



In [10]:
df2 = df.withColumn('revenue', F.col('totals.transactionRevenue')/1000000)

In [11]:
(df2
 .groupBy('date')
 .agg(
     F.count('*').alias('count'),
     F.sum('totals.visits'),
     F.sum('totals.pageviews'),
     F.sum('totals.transactions'),
     F.sum('revenue'))
 ).show()

+--------+-----+------------------+---------------------+------------------------+------------+
|    date|count|sum(totals.visits)|sum(totals.pageviews)|sum(totals.transactions)|sum(revenue)|
+--------+-----+------------------+---------------------+------------------------+------------+
|20170801| 2556|            2556.0|              10939.0|                    45.0|     8304.94|
+--------+-----+------------------+---------------------+------------------------+------------+



In [12]:
df2.groupBy('device.browser').agg(F.sum('totals.transactions').alias('total_transactions')).show()

+--------------------+------------------+
|             browser|total_transactions|
+--------------------+------------------+
|Mozilla Compatibl...|              null|
|   Internet Explorer|              null|
|             Firefox|               1.0|
|           YaBrowser|              null|
|              Safari|               3.0|
|       Nokia Browser|              null|
|             Coc Coc|              null|
|     Android Browser|              null|
|          Opera Mini|              null|
|     Android Webview|              null|
|     Safari (in-app)|              null|
|              Chrome|              41.0|
|          UC Browser|              null|
|                Edge|              null|
|               Opera|              null|
+--------------------+------------------+



In [13]:
# https://support.google.com/analytics/answer/4419694?hl=fr

In [14]:
(df2
 .groupBy('trafficSource.source')
 .agg(F.count('*').alias('total_visits'), 
      F.sum('totals.bounces').alias('total_bounces'),
      (100*F.sum('totals.bounces')/F.count('*')).alias('bounce_rate')
     )
 
 ).show()

+--------------------+------------+-------------+------------------+
|              source|total_visits|total_bounces|       bounce_rate|
+--------------------+------------+-------------+------------------+
|           quora.com|           6|          4.0| 66.66666666666667|
|         youtube.com|         180|        139.0| 77.22222222222223|
|productforums.goo...|           1|          1.0|             100.0|
|                bing|           1|         null|              null|
|     lm.facebook.com|           2|         null|              null|
|         m.baidu.com|           1|          1.0|             100.0|
|      l.facebook.com|           3|          3.0|             100.0|
| ph.search.yahoo.com|           1|          1.0|             100.0|
|          google.com|          12|          9.0|              75.0|
|     docs.google.com|           1|         null|              null|
|        sashihara.jp|           1|          1.0|             100.0|
|analytics.google.com|          57

In [15]:
(df2
 .filter('totals.transactions >= 1')
 .groupBy('fullVisitorId')
 .agg( F.sum('totals.pageviews').alias('total_pagesviews_per_user'),
      (F.sum('totals.pageviews') / F.count('*')).alias('avg_pageviews_per_user'),
       F.count('*').alias('nb_visits')
     )
 .orderBy(F.desc('nb_visits'))
).show()

+-------------------+-------------------------+----------------------+---------+
|      fullVisitorId|total_pagesviews_per_user|avg_pageviews_per_user|nb_visits|
+-------------------+-------------------------+----------------------+---------+
|7420300501523012460|                    133.0|                  66.5|        2|
|  03717831825503026|                     29.0|                  29.0|        1|
|4417987816137194149|                     14.0|                  14.0|        1|
| 454845221896711463|                     13.0|                  13.0|        1|
|5371617068925994598|                     23.0|                  23.0|        1|
| 351681171177883933|                     45.0|                  45.0|        1|
|0377506370870345639|                     24.0|                  24.0|        1|
|6403816382362616234|                     31.0|                  31.0|        1|
| 698297406423033664|                     16.0|                  16.0|        1|
|1996186587769697436|       

In [16]:
(df2
 .filter('totals.transactions = 0')
 .groupBy('fullVisitorId')
 .agg( F.sum('totals.pageviews').alias('total_pagesviews_per_user'),
      (F.sum('totals.pageviews') / F.count('*')).alias('avg_pageviews_per_user'),
       F.count('*').alias('nb_visits')
     )
 .orderBy(F.desc('nb_visits'))
).show()

+-------------+-------------------------+----------------------+---------+
|fullVisitorId|total_pagesviews_per_user|avg_pageviews_per_user|nb_visits|
+-------------+-------------------------+----------------------+---------+
+-------------+-------------------------+----------------------+---------+



In [17]:
df2.select('hits').printSchema()

root
 |-- hits: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- appInfo: struct (nullable = true)
 |    |    |    |-- exitScreenName: string (nullable = true)
 |    |    |    |-- landingScreenName: string (nullable = true)
 |    |    |    |-- screenDepth: string (nullable = true)
 |    |    |    |-- screenName: string (nullable = true)
 |    |    |-- contentGroup: struct (nullable = true)
 |    |    |    |-- contentGroup1: string (nullable = true)
 |    |    |    |-- contentGroup2: string (nullable = true)
 |    |    |    |-- contentGroup3: string (nullable = true)
 |    |    |    |-- contentGroup4: string (nullable = true)
 |    |    |    |-- contentGroup5: string (nullable = true)
 |    |    |    |-- contentGroupUniqueViews1: string (nullable = true)
 |    |    |    |-- contentGroupUniqueViews2: string (nullable = true)
 |    |    |    |-- contentGroupUniqueViews3: string (nullable = true)
 |    |    |    |-- previousContentGroup1: string (nulla

In [18]:
(df2
 .select('*',F.explode('hits').alias('hit'))
 .select('fullVisitorId','visitId','visitNumber','hit.page.pagePath','hit.hitNumber')
 .orderBy('fullVisitorId','visitId')
).show(10,40)

+-------------------+----------+-----------+----------------------------------------+---------+
|      fullVisitorId|   visitId|visitNumber|                                pagePath|hitNumber|
+-------------------+----------+-----------+----------------------------------------+---------+
|0004915997121163857|1501620292|          1|  /google+redesign/shop+by+brand/youtube|        1|
|0013182148727187801|1501580378|          1|                                   /home|        1|
|0013182148727187801|1501580378|          1|                                   /home|        3|
|0013182148727187801|1501580378|          1|/google+redesign/bags/backpacks/water...|        5|
|0013182148727187801|1501580378|          1|                                   /home|        6|
|0013182148727187801|1501580378|          1|/google+redesign/drinkware/google+17o...|        7|
|0013182148727187801|1501580378|          1|                                   /home|        4|
|0013182148727187801|1501580378|        

In [19]:
# temps ecoule depuis la derniere visite?

In [20]:
# Quel est le nombre moyen d'interactions de l'utilisateur avant un achat ?

In [21]:
(df2
 .select(F.explode('hits').alias('hit'))
 .select(F.explode(F.col('hit').product).alias('product'))
 .select(F.col('product'))
 
).printSchema()

root
 |-- product: struct (nullable = true)
 |    |-- customDimensions: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- customMetrics: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- isClick: boolean (nullable = true)
 |    |-- isImpression: boolean (nullable = true)
 |    |-- localProductPrice: string (nullable = true)
 |    |-- localProductRevenue: string (nullable = true)
 |    |-- productBrand: string (nullable = true)
 |    |-- productListName: string (nullable = true)
 |    |-- productListPosition: string (nullable = true)
 |    |-- productPrice: string (nullable = true)
 |    |-- productQuantity: string (nullable = true)
 |    |-- productRevenue: string (nullable = true)
 |    |-- productSKU: string (nullable = true)
 |    |-- productVariant: string (nullable = true)
 |    |-- v2ProductCategory: string (nullable = true)
 |    |-- v2ProductName: string (nullable = true)



In [22]:
(df2
 .select(F.explode('hits').alias('hit'))
 .select(F.explode(F.col('hit').product).alias('product'))
 .select('product.v2ProductCategory','product.v2ProductName')
 .distinct()
).show()

+--------------------+--------------------+
|   v2ProductCategory|       v2ProductName|
+--------------------+--------------------+
|   Home/Electronics/|   Google Flashlight|
|Home/Shop by Bran...|Android Wool Heat...|
|          Home/Bags/|Google Canvas Tot...|
| Home/Apparel/Kid's/|Google Infant Sho...|
|Home/Apparel/Wome...|Google Women's Co...|
|Home/Apparel/Kid'...|Google Infant Sho...|
| Home/Apparel/Men's/|Google Men's Colo...|
| Home/Shop by Brand/|YouTube Men's Vin...|
|Home/Accessories/...|Google Car Clip P...|
|Home/Apparel/Men'...|Google Men's 100%...|
|                 Fun|  Google Luggage Tag|
|Home/Apparel/Men'...|Google Men's  Zip...|
|Home/Apparel/Wome...|Google Women's We...|
| Home/Shop by Brand/|      Windup Android|
|Home/Apparel/Men'...|Google Men's Micr...|
|Home/Shop by Bran...|22 oz Android Bottle|
| Home/Apparel/Men's/|Google Men's 100%...|
|Home/Office/Noteb...| Google RFID Journal|
|Home/Office/Writi...|Ballpoint Stylus Pen|
|Home/Bags/Shoppin...|Collapsibl

In [23]:
(df2
 .select(F.col('device.browser').alias('browser'),F.explode('hits').alias('hit'))
 .select('browser',F.explode(F.col('hit').product).alias('product'))
 .select('browser','product.v2ProductCategory','product.v2ProductName')

).show()

+-------+--------------------+--------------------+
|browser|   v2ProductCategory|       v2ProductName|
+-------+--------------------+--------------------+
| Chrome|   Home/Electronics/|Electronics Acces...|
| Chrome|   Home/Electronics/|   Google Flashlight|
| Chrome|   Home/Electronics/|Micro Wireless Ea...|
| Chrome|   Home/Electronics/|Google Car Clip P...|
| Chrome|   Home/Electronics/|Keyboard DOT Sticker|
| Chrome|   Home/Electronics/|Google Device Hol...|
| Chrome|   Home/Electronics/| Google Device Stand|
| Chrome|   Home/Electronics/|Google 2200mAh Mi...|
| Chrome|   Home/Electronics/|Google 4400mAh Po...|
| Chrome|   Home/Electronics/|Basecamp Explorer...|
| Chrome|   Home/Electronics/|   Rocket Flashlight|
| Chrome|   Home/Electronics/|Plastic Sliding F...|
| Chrome|Home/Apparel/Men'...|Google Tri-blend ...|
| Chrome|Home/Apparel/Men'...|Google Men's  Zip...|
| Chrome|Home/Apparel/Men'...|Google Men's Wate...|
| Chrome|Home/Apparel/Men'...|Google Men's Airf...|
| Chrome|Hom

In [24]:
(df2
 .select(F.col('device.browser').alias('browser'),F.explode('hits').alias('hit'))
 .select('browser',F.explode(F.col('hit').product).alias('product'))
 .select('browser','product.v2ProductCategory','product.v2ProductName')
 .groupBy('v2ProductName')
 .pivot('browser')
 .count()
 .fillna(0)
).show()

+--------------------+---------------+---------------+------+-------+----+-------+-----------------+------------------------+-------------+-----+----------+------+---------------+----------+---------+
|       v2ProductName|Android Browser|Android Webview|Chrome|Coc Coc|Edge|Firefox|Internet Explorer|Mozilla Compatible Agent|Nokia Browser|Opera|Opera Mini|Safari|Safari (in-app)|UC Browser|YaBrowser|
+--------------------+---------------+---------------+------+-------+----+-------+-----------------+------------------------+-------------+-----+----------+------+---------------+----------+---------+
|  Gift Card- $100.00|              0|              0|    17|      0|   0|      0|                0|                       0|            0|    0|         0|     0|              0|         0|        0|
|Google Men's Perf...|              0|              0|   313|      0|   1|     10|                1|                       0|            0|    1|         0|    25|              0|         1|      

In [25]:
df2.groupBy('device.browser').count().orderBy(F.desc('count'))

DataFrame[browser: string, count: bigint]

In [26]:
new_browser = F.when(F.col('device.browser').isin('Chrome','Safari','Firefox','Internet Explorer'),F.col('device.browser')).otherwise('Other')

In [27]:
(df2
 .select(new_browser.alias('browser'),F.explode('hits').alias('hit'))
 .select('browser',F.explode(F.col('hit').product).alias('product'))
 .select('browser','product.v2ProductCategory','product.v2ProductName')
 .groupBy('v2ProductName')
 .pivot('browser')
 .count()
 .fillna(0)
).show(10,40)

+----------------------------------------+------+-------+-----------------+-----+------+
|                           v2ProductName|Chrome|Firefox|Internet Explorer|Other|Safari|
+----------------------------------------+------+-------+-----------------+-----+------+
|                      Gift Card- $100.00|    17|      0|                0|    0|     0|
|Google Men's Performance Full Zip Jac...|   313|     10|                1|    4|    25|
|Android Women's Short Sleeve Badge Te...|    46|      3|                2|    0|     7|
|               Google Executive Umbrella|    66|      4|                0|    0|     3|
|     Google Women's Scoop Neck Tee Black|   128|      1|                1|    0|     8|
|    Android Infant Short Sleeve Tee Pink|    34|      1|                0|    0|     0|
|                     Red Shine 15 oz Mug|   173|     11|                1|    4|    13|
|          Google Stylus Pen w/ LED Light|    60|      6|                0|    2|    10|
|YouTube Men's Short 

In [28]:
df3 = (df2
 .select(new_browser.alias('browser'),F.explode('hits').alias('hit'))
 .select('browser',F.explode(F.col('hit').product).alias('product'))
 .select('browser','product.v2ProductCategory','product.v2ProductName')
 .groupBy('v2ProductName')
 .pivot('browser')
 .count()
 .fillna(0)
)

In [29]:
df3.filter('Firefox > Safari').show()

+--------------------+------+-------+-----------------+-----+------+
|       v2ProductName|Chrome|Firefox|Internet Explorer|Other|Safari|
+--------------------+------+-------+-----------------+-----+------+
|Google Executive ...|    66|      4|                0|    0|     3|
|Android Infant Sh...|    34|      1|                0|    0|     0|
|Google Men's 100%...|    35|      3|                0|    0|     2|
|Waze Mobile Phone...|    72|      3|                0|    4|     1|
|Google Men's Vint...|     0|      1|                0|    0|     0|
| Android Luggage Tag|   225|     16|                1|    4|    14|
|Waze Mood Origina...|    95|      4|                1|    1|     3|
|Satin Black Ballp...|    96|     15|                0|    2|    10|
| 7&quot; Dog Frisbee|    36|      2|                0|    0|     1|
|Metal Texture Rol...|    82|     13|                0|    2|    10|
|Google G Noise-re...|   161|     10|                1|    5|     5|
|Google Car Clip P...|   159|     

In [30]:
df3.filter('Firefox < Safari').show()

+--------------------+------+-------+-----------------+-----+------+
|       v2ProductName|Chrome|Firefox|Internet Explorer|Other|Safari|
+--------------------+------+-------+-----------------+-----+------+
|Google Men's Perf...|   313|     10|                1|    4|    25|
|Android Women's S...|    46|      3|                2|    0|     7|
|Google Women's Sc...|   128|      1|                1|    0|     8|
| Red Shine 15 oz Mug|   173|     11|                1|    4|    13|
|Google Stylus Pen...|    60|      6|                0|    2|    10|
|YouTube Men's Sho...|   503|     44|               27|   63|   227|
|Google Women's Sh...|    14|      1|                0|    0|     3|
|Android RFID Journal|   263|     12|                4|    7|    27|
|8 pc Android Stic...|   286|      7|                7|    6|    41|
|Google Women's 3/...|   145|      4|                1|    0|    12|
| Google Device Stand|   360|     13|                6|   10|    53|
|Micro Wireless Ea...|   247|     

In [32]:
df.write.save('ga_sample.parquet')

## Parquet vs Json

In [38]:
%%time
df_json = spark.read.json('ga_sample.json.gz')
print(df_json.count())

2556
CPU times: user 0 ns, sys: 10 ms, total: 10 ms
Wall time: 733 ms


In [39]:
%%time
df_parquet = spark.read.parquet('ga_sample.parquet')
print(df_parquet.count())

2556
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 299 ms


## RDDs

In [40]:
rdd = spark.sparkContext.textFile('ga_sample.json.gz')

In [41]:
rdd.first()

'{"visitNumber":"1","visitId":"1501583974","visitStartTime":"1501583974","date":"20170801","totals":{"visits":"1","hits":"1","pageviews":"1","bounces":"1","newVisits":"1","sessionQualityDim":"1"},"trafficSource":{"campaign":"(not set)","source":"(direct)","medium":"(none)","adwordsClickInfo":{"criteriaParameters":"not available in demo dataset"}},"device":{"browser":"Chrome","browserVersion":"not available in demo dataset","browserSize":"not available in demo dataset","operatingSystem":"Android","operatingSystemVersion":"not available in demo dataset","isMobile":true,"mobileDeviceBranding":"not available in demo dataset","mobileDeviceModel":"not available in demo dataset","mobileInputSelector":"not available in demo dataset","mobileDeviceInfo":"not available in demo dataset","mobileDeviceMarketingName":"not available in demo dataset","flashVersion":"not available in demo dataset","language":"not available in demo dataset","screenColors":"not available in demo dataset","screenResolution

In [43]:
import json

In [44]:
parsed = rdd.map(json.loads)

In [45]:
parsed.first()

{'visitNumber': '1',
 'visitId': '1501583974',
 'visitStartTime': '1501583974',
 'date': '20170801',
 'totals': {'visits': '1',
  'hits': '1',
  'pageviews': '1',
  'bounces': '1',
  'newVisits': '1',
  'sessionQualityDim': '1'},
 'trafficSource': {'campaign': '(not set)',
  'source': '(direct)',
  'medium': '(none)',
  'adwordsClickInfo': {'criteriaParameters': 'not available in demo dataset'}},
 'device': {'browser': 'Chrome',
  'browserVersion': 'not available in demo dataset',
  'browserSize': 'not available in demo dataset',
  'operatingSystem': 'Android',
  'operatingSystemVersion': 'not available in demo dataset',
  'isMobile': True,
  'mobileDeviceBranding': 'not available in demo dataset',
  'mobileDeviceModel': 'not available in demo dataset',
  'mobileInputSelector': 'not available in demo dataset',
  'mobileDeviceInfo': 'not available in demo dataset',
  'mobileDeviceMarketingName': 'not available in demo dataset',
  'flashVersion': 'not available in demo dataset',
  'langu

### Query 1

In [46]:
# DataFrame
df.groupBy('device.browser').count().orderBy(F.desc('count')).show(5)

+-----------------+-----+
|          browser|count|
+-----------------+-----+
|           Chrome| 1900|
|           Safari|  397|
|          Firefox|  101|
|Internet Explorer|   54|
|             Edge|   23|
+-----------------+-----+
only showing top 5 rows



In [49]:
# RDD
(parsed
 .map(lambda x: (x['device']['browser'],1))
 .reduceByKey(lambda x,y: x+y)
 .top(5, key = lambda x : x[1])
)

[('Chrome', 1900),
 ('Safari', 397),
 ('Firefox', 101),
 ('Internet Explorer', 54),
 ('Edge', 23)]

### Query 2

In [51]:
# DataFrame
(df2
 .select(F.explode('hits').alias('hit'))
 .select(F.explode(F.col('hit').product).alias('product'))
 .groupBy('product.v2ProductCategory')
 .count()
 .orderBy(F.desc('count'))
).show(5,truncate=False)

+----------------------------------+-----+
|v2ProductCategory                 |count|
+----------------------------------+-----+
|Home/Shop by Brand/YouTube/       |9386 |
|Home/Apparel/Men's/Men's-T-Shirts/|4874 |
|(not set)                         |3022 |
|Home/Electronics/                 |2215 |
|Home/Bags/                        |2101 |
+----------------------------------+-----+
only showing top 5 rows



In [52]:
# RDD
(parsed
 .flatMap(lambda x: [(product['v2ProductCategory'],1) for hit in x['hits'] for product in hit['product']])
 .reduceByKey(lambda x,y: x+y)
 .top(5, key = lambda x : x[1])
)

[('Home/Shop by Brand/YouTube/', 9386),
 ("Home/Apparel/Men's/Men's-T-Shirts/", 4874),
 ('(not set)', 3022),
 ('Home/Electronics/', 2215),
 ('Home/Bags/', 2101)]

### Query 3 

In [54]:
# DataFrame
(df2
 .select(new_browser.alias('browser'),F.explode('hits').alias('hit'))
 .select('browser',F.explode(F.col('hit').product).alias('product'))
 .select('browser','product.v2ProductCategory','product.v2ProductName')
 .groupBy('v2ProductName')
 .pivot('browser')
 .count()
 .fillna(0)
).show(10,40)

+----------------------------------------+------+-------+-----------------+-----+------+
|                           v2ProductName|Chrome|Firefox|Internet Explorer|Other|Safari|
+----------------------------------------+------+-------+-----------------+-----+------+
|                      Gift Card- $100.00|    17|      0|                0|    0|     0|
|Google Men's Performance Full Zip Jac...|   313|     10|                1|    4|    25|
|Android Women's Short Sleeve Badge Te...|    46|      3|                2|    0|     7|
|               Google Executive Umbrella|    66|      4|                0|    0|     3|
|     Google Women's Scoop Neck Tee Black|   128|      1|                1|    0|     8|
|    Android Infant Short Sleeve Tee Pink|    34|      1|                0|    0|     0|
|                     Red Shine 15 oz Mug|   173|     11|                1|    4|    13|
|          Google Stylus Pen w/ LED Light|    60|      6|                0|    2|    10|
|YouTube Men's Short 

In [55]:
# RDD à faire 

## Hive 

In [1]:
from os.path import expanduser, join, abspath
from pyspark.sql import SparkSession
from pyspark.sql import Row

warehouse_location = abspath('spark-warehouse')

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .enableHiveSupport() \
    .getOrCreate()

In [11]:
spark.sql("select date from json.`ga_sample.json.gz`").show(5)

+--------+
|    date|
+--------+
|20170801|
|20170801|
|20170801|
|20170801|
|20170801|
+--------+
only showing top 5 rows



In [10]:
spark.sql("select date from parquet.`ga_sample.parquet`").show(5)

+--------+
|    date|
+--------+
|20170801|
|20170801|
|20170801|
|20170801|
|20170801|
+--------+
only showing top 5 rows



In [4]:
hiveDF = spark.sql("select * from json.`ga_sample.json.gz`")

In [5]:
hiveDF.printSchema()

root
 |-- channelGrouping: string (nullable = true)
 |-- customDimensions: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- index: string (nullable = true)
 |    |    |-- value: string (nullable = true)
 |-- date: string (nullable = true)
 |-- device: struct (nullable = true)
 |    |-- browser: string (nullable = true)
 |    |-- browserSize: string (nullable = true)
 |    |-- browserVersion: string (nullable = true)
 |    |-- deviceCategory: string (nullable = true)
 |    |-- flashVersion: string (nullable = true)
 |    |-- isMobile: boolean (nullable = true)
 |    |-- language: string (nullable = true)
 |    |-- mobileDeviceBranding: string (nullable = true)
 |    |-- mobileDeviceInfo: string (nullable = true)
 |    |-- mobileDeviceMarketingName: string (nullable = true)
 |    |-- mobileDeviceModel: string (nullable = true)
 |    |-- mobileInputSelector: string (nullable = true)
 |    |-- operatingSystem: string (nullable = true)
 |    |-- operati

In [9]:
df_json = spark.read.json('ga_sample.json.gz')
df_json.registerTempTable("ga_session")

spark.sql("select date from ga_session").show(5)

+--------+
|    date|
+--------+
|20170801|
|20170801|
|20170801|
|20170801|
|20170801|
+--------+
only showing top 5 rows

